In [6]:
import torch
from utils.utils import HadamardTransform
from utils.hadamard_utils import matmul_hadU_cuda, get_hadK
import math
import numpy as np
import struct

In [7]:
# Function to get the bit pattern of a floating-point number
def float_to_hex(f):
    # Pack the float into 4 bytes (32-bit float) or 8 bytes (64-bit float)
    packed = struct.pack('>f', f)  # Use '>f' for 32-bit float, '>d' for 64-bit double
    # Unpack as an integer to get the bit pattern
    hex_value = ''.join(f'{byte:02x}' for byte in packed)
    return hex_value

In [16]:
q = torch.rand(2,2,11008, dtype=torch.float32, device='cuda')
q_pad = torch.concat([q, torch.zeros(q.shape[0], q.shape[1], 5376, dtype=q.dtype, device="cuda")], dim=-1)

# q = torch.rand(2,2,11008, dtype=torch.float32)
# q_pad = torch.concat([q, torch.zeros(q.shape[0], q.shape[1], 5376, dtype=q.dtype)], dim=-1)

np.log2([11008])
print(2**14)
print(16384-11008)

dtype = q.dtype
q_ref = (HadamardTransform.apply(q.float()) / math.sqrt(q.shape[-1])).to(dtype)
q_evl = (HadamardTransform.apply(q_pad.float()) / math.sqrt(q.shape[-1])).to(dtype)

print((q_ref-q_evl[...,:11008]).abs().mean())
print(((q_ref-q_evl[...,:11008]).abs()/(q_ref.abs() + 1e-6)).abs().mean())

16384
5376
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')


In [76]:
n1, n2, n3 = 2, 2, 4*172
q = torch.zeros(n1*n2*n3, dtype=torch.float32)
for i in range(n1*n2*n3):
  q[i] = i/(n1*n2*n3)
q = q.reshape(n1, n2, n3)


had_K, K = get_hadK(q.shape[-1])
q_had = matmul_hadU_cuda(q, had_K, K)

print(had_K)
print(K)
print(q_had)

data = q_had.numpy()
data.tofile('./hadamard_test/hadamard_transform.bin')

tensor([[ 1., -1., -1.,  ..., -1., -1.,  1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.],
        [-1., -1.,  1.,  ...,  1., -1., -1.],
        ...,
        [ 1.,  1., -1.,  ...,  1., -1., -1.],
        [ 1.,  1.,  1.,  ..., -1.,  1., -1.],
        [-1.,  1.,  1.,  ..., -1., -1.,  1.]])
172
tensor([[[ 5.7852e-02, -2.2165e-04, -4.4332e-04,  ..., -1.6624e-04,
          -3.3248e-04, -1.7198e-10],
         [ 3.6285e-01, -2.2166e-04, -4.4331e-04,  ..., -1.6624e-04,
          -3.3250e-04,  3.4086e-09]],

        [[ 6.6785e-01, -2.2165e-04, -4.4331e-04,  ..., -1.6623e-04,
          -3.3252e-04, -1.1362e-08],
         [ 9.7284e-01, -2.2165e-04, -4.4331e-04,  ..., -1.6623e-04,
          -3.3252e-04, -1.1362e-08]]])


In [77]:
q_had_ref = q_had.reshape(-1, n3).to(torch.float32)

In [78]:
for i in range(q_had_ref.shape[0]):
    for j in range(q_had_ref.shape[1]):
        hex_value = float_to_hex(q_had_ref[i, j].item())
        print(f"0x{hex_value}", end=" ")
    print()

0x3d6cf635 0xb9686bec 0xb9e86cee 0xaf22422c 0x3d4d2f77 0xb9686bea 0xb9e86cde 0x303fd7dd 0x3d7b7ce7 0xb9686be8 0xb9e86cfc 0xafd13a64 0x3d94e53a 0xb9686c59 0xb9e86d32 0x3045f171 0x3d8501d4 0xb9686cbd 0xb9e86cbe 0xafe38722 0x3d6a3cf1 0xb9686cd9 0xb9e86c74 0x2de07a60 0x3d4a761f 0xb9686cc1 0xb9e86cb8 0x2fd9c499 0x3d2aaf66 0xb9686c56 0xb9e86d24 0xb036b17c 0x3d0ae890 0xb9686bee 0xb9e86cfe 0x2fc9e87c 0x3d393623 0xb9686bf6 0xb9e86cea 0xb01ee74c 0x3d196f4e 0xb9686be6 0xb9e86ce2 0x2f5f4204 0x3d47bce1 0xb9686c36 0xb9e86c4a 0xb041102e 0x3d760a69 0xb9686c1e 0xb9e86c16 0xaf160f00 0x3d56439b 0xb9686c1e 0xb9e86bf6 0xaef6700c 0x3d824898 0xb9686bae 0xb9e86c26 0xb09dd606 0x3d64ca6f 0xb9686b3a 0xb9e86be6 0x2f64234a 0x3d4503a9 0xb9686b02 0xb9e86c22 0x302ceef2 0x3d253cd0 0xb9686b56 0xb9e86c42 0x2f8ce8a0 0x3d538a67 0xb9686c12 0xb9e86c06 0x2e8151a0 0x3d33c38f 0xb9686c16 0xb9e86c2a 0xae8151a0 0x3d621123 0xb9686c02 0xb9e86bd2 0xaf4e2d96 0x3d882f58 0xb9686c36 0xb9e86b82 0xb0856fae 0x3d9f5616 0xb9686bda 0xb9e86b3a

In [4]:
a = torch.tensor([1100.0, -1110.1, 7.6, 8.4], dtype=torch.float32)

In [5]:
a.to(torch.int8)

tensor([ 76, -86,   7,   8], dtype=torch.int8)